# Import

In [14]:
import numpy as np
import pandas as pd

# Data preporation

In [15]:
# Load previous day data
file_name = 'release/covid19-russia-cases.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
5505,2020-06-07,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,115.0,0.0,14.0,3346.0,20.0,1126.0
5506,2020-06-07,Чеченская Республика,Chechen Republic,95.0,19.0,0.0,2.0,1345.0,14.0,990.0
5507,2020-06-07,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,4.0,0.0,2.0,106.0,1.0,65.0
5508,2020-06-07,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,73.0,0.0,53.0,2694.0,11.0,1429.0
5509,2020-06-07,Ярославская область,Yaroslavl region,76.0,70.0,0.0,7.0,3646.0,15.0,1176.0


In [16]:
# Read update data
# Result of data_parsing.ipynb
file_name = 'covid19-russia-cases-upd.csv'
upd_df = pd.read_csv(file_name)
upd_df.sort_values('Confirmed', ascending=False).head()

,Region/City,Confirmed,Date,Deaths,Recovered,Day-Confirmed,Day-Deaths,Day-Recovered
29,Москва,197018.0,2020-06-08,2970.0,104347.0,2001.0,51.0,1633.0
30,Московская область,44983.0,2020-06-08,578.0,12031.0,751.0,11.0,569.0
63,Санкт-Петербург,18835.0,2020-06-08,392.0,7915.0,326.0,15.0,156.0
32,Нижегородская область,12442.0,2020-06-08,140.0,4878.0,305.0,4.0,41.0
66,Свердловская область,7668.0,2020-06-08,43.0,3273.0,249.0,2.0,34.0


In [17]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
upd_df['Region/City'] = upd_df['Region/City'].astype('str').str.strip('\u200b')

# Updating

In [18]:
def upd(row):
    reg = row['Region/City']
    
    row['Confirmed'] = rus_df[rus_df['Region/City'] == reg]['Confirmed'].max() + row['Day-Confirmed']
    row['Deaths'] = rus_df[rus_df['Region/City'] == reg]['Deaths'].max() + row['Day-Deaths']
    row['Recovered'] = rus_df[rus_df['Region/City'] == reg]['Recovered'].max() + row['Day-Recovered']
    
    row['Confirmed'] = row['Day-Confirmed'] if np.isnan(row['Confirmed']) else row['Confirmed']
    row['Deaths'] = row['Day-Deaths'] if np.isnan(row['Deaths']) else row['Deaths']
    row['Recovered'] = row['Day-Recovered'] if np.isnan(row['Recovered']) else row['Recovered']
    
    return row.drop('Region/City')

# If cumulative values unknown
# upd_df = upd_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [19]:
# Date to datetime
upd_df['Date'] = pd.to_datetime(upd_df['Date'])
rus_df['Date'] = pd.to_datetime(rus_df['Date'])

In [20]:
# Region names translation
# from googletrans import Translator
# translator = Translator()

# def translate(rus):
#     eng = translator.translate(rus, src='ru', dest='en')
#     return eng.text

# upd_df['Region/City-Eng'] = upd_df['Region/City'].apply(translate)

In [21]:
# Add region ID and Eng columns
regions_df = pd.read_csv('release/regions-info.csv')

right = regions_df[['Region_ID', 'Region', 'Region_eng']]
upd_df = upd_df.merge(right, left_on='Region/City', right_on='Region', how='left')
upd_df['Region/City-Eng'] = upd_df.Region_eng
upd_df = upd_df[['Date', 'Region/City', 'Region/City-Eng', 'Region_ID', 'Day-Confirmed', 'Day-Deaths', 'Day-Recovered', 'Confirmed', 'Deaths', 'Recovered']]

In [22]:
rus_df = pd.concat([rus_df, upd_df]).reset_index(drop=True)
rus_df.tail(10)

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
5583,2020-06-08,Тульская область,Tula region,71.0,114.0,0.0,7.0,4734.0,49.0,2441.0
5584,2020-06-08,Тюменская область,Tyumen region,72.0,39.0,0.0,10.0,1947.0,12.0,1067.0
5585,2020-06-08,Удмуртская Республика,Udmurt republic,18.0,27.0,0.0,0.0,963.0,20.0,467.0
5586,2020-06-08,Ульяновская область,Ulyanovsk region,73.0,97.0,2.0,0.0,3265.0,16.0,954.0
5587,2020-06-08,Хабаровский край,Khabarovsk region,27.0,59.0,0.0,13.0,2596.0,17.0,1314.0
5588,2020-06-08,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,129.0,1.0,60.0,3475.0,21.0,1186.0
5589,2020-06-08,Челябинская область,Chelyabinsk region,74.0,128.0,3.0,80.0,3847.0,31.0,1801.0
5590,2020-06-08,Чеченская Республика,Chechen Republic,95.0,20.0,1.0,3.0,1365.0,15.0,993.0
5591,2020-06-08,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,83.0,0.0,46.0,2777.0,11.0,1475.0
5592,2020-06-08,Ярославская область,Yaroslavl region,76.0,68.0,0.0,4.0,3714.0,15.0,1180.0


In [23]:
# Negative values fixing
# Negative values can appear if some cases was removed from statistic
def negval_fix(reg):
    last = reg.iloc[-1]
    diff_c = last['Day-Confirmed']*(-1) if last['Day-Confirmed'] < 0 else 0
    diff_d = last['Day-Deaths']*(-1) if last['Day-Deaths'] < 0 else 0
    diff_r = last['Day-Recovered']*(-1) if last['Day-Recovered'] < 0 else 0
    
    last['Day-Confirmed'] += diff_c
    last['Day-Deaths']    += diff_d
    last['Day-Recovered'] += diff_r
    reg.iloc[-1] = last
    
    if diff_c + diff_d + diff_r > 0:
        print(f"Conf {diff_c} Death {diff_d} Rec {diff_r} {last['Region/City']}")
        prev = reg.iloc[-2]
        prev['Confirmed'] -= diff_c
        prev['Deaths']    -= diff_d
        prev['Recovered'] -= diff_r
        reg.iloc[-2] = prev
    
    return reg
    
rus_df = rus_df.groupby('Region/City').apply(negval_fix).reset_index(drop=True)

In [24]:
# Manual fixes if needed
# rus_df.loc[321, 'Confirmed'] = 5
# rus_df.loc[359, 'Day-Confirmed'] = 0


# rus_df.loc[3775, 'Deaths'] = 7
# rus_df.loc[3775, 'Day-Deaths'] = 0

# Data checking

In [25]:
rus_sum = rus_df.loc[rus_df['Region/City'] != 'Diamond Princess']

print('Russia Confirmed:', rus_sum.groupby('Region/City')['Confirmed'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', rus_sum.groupby('Region/City')['Deaths'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', rus_sum.groupby('Region/City')['Recovered'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Recovered'].sum().sum())

rus_regs = rus_sum.groupby('Region/City')['Confirmed'].max().reset_index()
rus_regs['Confirmed-ByDay'] = rus_sum.groupby('Region/City')['Day-Confirmed'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Confirmed'] != rus_regs['Confirmed-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Deaths'].max().reset_index()
rus_regs['Deaths-ByDay'] = rus_sum.groupby('Region/City')['Day-Deaths'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Deaths'] != rus_regs['Deaths-ByDay']])

# rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 476768.0 476768.0
Russia Deaths: 5997.0 5997.0
Russia Recovered: 230938.0 230938.0
Empty DataFrame
Columns: [Region/City, Confirmed, Confirmed-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Recovered, Recovered-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Deaths, Deaths-ByDay]
Index: []


# Data saving

In [26]:
file_name = 'release/covid19-russia-cases.csv'
rus_df.to_csv(file_name, index=False)